In [1]:
! pip install datasets transformers seqeval peft

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 3.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.6/251.6 kB 18.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 12.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.4/309.4 kB 19.9 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.10

In [44]:
!pip install evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 2.0 MB/s eta 0:00:00


In [2]:
model_checkpoint = "distilbert/distilbert-base-multilingual-cased"
batch_size = 16

In [5]:
from datasets import load_dataset, load_metric

datasets = load_dataset("unimelb-nlp/wikiann", "fa")

In [12]:
from datasets import ClassLabel, Sequence
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)

    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
        elif isinstance(typ, Sequence) and isinstance(typ.feature, ClassLabel):
            df[column] = df[column].transform(lambda x: [typ.feature.names[i] for i in x])
    display(HTML(df.to_html()))

show_random_elements(datasets["train"])

,tokens,ner_tags,langs,spans
0,"[بزرگراه, اشرفی, اصفهانی]","[B-ORG, I-ORG, I-ORG]","[fa, fa, fa]",[ORG: بزرگراه اشرفی اصفهانی]
1,"[تغییرمسیر, نخستین, پیروان, بهاءالله]","[O, B-PER, I-PER, I-PER]","[fa, fa, fa, fa]",[PER: نخستین پیروان بهاءالله]
2,"[تغییرمسیر, ماهیان, (, علی‌آباد, )]","[O, B-LOC, I-LOC, I-LOC, I-LOC]","[fa, fa, fa, fa, fa]",[LOC: ماهیان ( علی‌آباد )]
3,"['', گویان, '', ']","[O, B-LOC, O, O]","[fa, fa, fa, fa]",[LOC: گویان]
4,"[تغییرمسیر, پیچ‌ده, (, چالوس, )]","[O, B-LOC, I-LOC, I-LOC, I-LOC]","[fa, fa, fa, fa, fa]",[LOC: پیچ‌ده ( چالوس )]
5,"[مرکز, بین‌المللی, فیزیک, نظری]","[B-ORG, I-ORG, I-ORG, I-ORG]","[fa, fa, fa, fa]",[ORG: مرکز بین‌المللی فیزیک نظری]
6,"[تغییرمسیر, شهرستان, گرین،, کارولینای, شمالی]","[O, B-LOC, I-LOC, I-LOC, I-LOC]","[fa, fa, fa, fa, fa]",[LOC: شهرستان گرین، کارولینای شمالی]
7,"[لایپزیگ, /, هاله, (, شهر, )]","[B-LOC, O, B-LOC, I-LOC, I-LOC, I-LOC]","[fa, fa, fa, fa, fa, fa]","[LOC: لایپزیگ, LOC: هاله ( شهر )]"
8,"[تغییرمسیر, نوناتو, (, بازیکن, فوتبال, )]","[O, B-PER, I-PER, I-PER, I-PER, I-PER]","[fa, fa, fa, fa, fa, fa]",[PER: نوناتو ( بازیکن فوتبال )]
9,"[تغییرمسیر, امیدآباد, (, فارسان, )]","[O, B-LOC, I-LOC, I-LOC, I-LOC]","[fa, fa, fa, fa, fa]",[LOC: امیدآباد ( فارسان )]


In [13]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/466 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

In [22]:
label_all_tokens = True

def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(examples["tokens"], truncation=True, is_split_into_words=True)

    labels = []
    for i, label in enumerate(examples["ner_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            # Special tokens have a word id that is None. We set the label to -100 so they are automatically
            # ignored in the loss function.
            if word_idx is None:
                label_ids.append(-100)
            # We set the label for the first token of each word.
            elif word_idx != previous_word_idx:
                label_ids.append(label[word_idx])
            # For the other tokens in a word, we set the label to either the current label or -100, depending on
            # the label_all_tokens flag.
            else:
                label_ids.append(label[word_idx] if label_all_tokens else -100)
            previous_word_idx = word_idx

        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

In [23]:
tokenize_and_align_labels(datasets['train'][:5])

{'input_ids': [[101, 37260, 73261, 15747, 77010, 16506, 113, 770, 35067, 35418, 114, 102], [101, 112, 112, 112, 15422, 769, 10700, 15470, 112, 112, 112, 102], [101, 37260, 73261, 15747, 14043, 47291, 19171, 12710, 113, 14043, 101736, 10388, 114, 102], [101, 84497, 11626, 768, 43884, 33229, 22964, 784, 11884, 42910, 10502, 12616, 15470, 100344, 67453, 12190, 15450, 20515, 22161, 22114, 11892, 119, 102], [101, 10748, 15193, 788, 52891, 38764, 10278, 10239, 20987, 138, 22786, 10219, 102]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]], 'labels': [[-100, 0, 0, 0, 5, 5, 6, 6, 6, 6, 6, -100], [-100, 0, 0, 0, 5, 6, 6, 6, 0, 0, 0, -100], [-100, 0, 0, 0, 5, 5, 5, 5, 6, 6, 6, 6, 6, -100], [-100, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 4, 4, 4, 3, 4, 0, 0, -100], [-100, 1, 1, 2, 2, 2, 2, 0, 0,

In [24]:
train_dataset = datasets["train"].shuffle(seed=42).select(range(500))
eval_dataset = datasets["train"].shuffle(seed=42).select(range(500, 600))

In [25]:
train_dataset = train_dataset.map(tokenize_and_align_labels, batched=True)
eval_dataset = eval_dataset.map(tokenize_and_align_labels, batched=True)

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

In [26]:
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer)

In [59]:
label_feature = datasets["train"].features["ner_tags"]
unique_labels = set(label for example in datasets["train"] for label in example["ner_tags"])
label_list = list(unique_labels)
print(label_list)

[0, 1, 2, 3, 4, 5, 6]


In [41]:
from transformers import AutoModelForTokenClassification
from peft import get_peft_model, LoraConfig, TaskType

# Load the model
model = AutoModelForTokenClassification.from_pretrained(model_checkpoint, num_labels=len(label_list))

print(model)

# Define LoRA configuration with target modules
lora_config = LoraConfig(
    task_type=TaskType.TOKEN_CLS,
    inference_mode=False,
    r=8,
    lora_alpha=32,
    lora_dropout=0.1,
    target_modules=["q_lin", "k_lin","v_lin"]
)

# Apply LoRA to the model
model = get_peft_model(model, lora_config)


Some weights of DistilBertForTokenClassification were not initialized from the model checkpoint at distilbert/distilbert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


DistilBertForTokenClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
   

In [62]:
metric = load_metric("seqeval")

labels = [label_list[i] for i in example["ner_tags"]]
metric.compute(predictions=[labels], references=[labels])

{'PER': {'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'number': 1},
 'overall_precision': 1.0,
 'overall_recall': 1.0,
 'overall_f1': 1.0,
 'overall_accuracy': 1.0}

In [65]:
import numpy as np
from evaluate import load

metric = load("seqeval")
label_list = datasets["train"].features["ner_tags"].feature.names
print( label_list)
def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    # Remove ignored index (special tokens)
    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC']


In [68]:
from transformers import TrainingArguments, Trainer

args = TrainingArguments(
    output_dir="./results",
    eval_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=50,
    weight_decay=0.01,
)

In [69]:
trainer = Trainer(
    model,
    args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [70]:
trainer.train()

Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,No log,1.371122,0.072816,0.060241,0.065934,0.522538
2,No log,1.322586,0.098592,0.084337,0.090909,0.570952
3,No log,1.274109,0.155660,0.132530,0.143167,0.611018
4,No log,1.225824,0.181818,0.144578,0.161074,0.641903
5,No log,1.176556,0.195531,0.140562,0.163551,0.663606
6,No log,1.126163,0.197452,0.124498,0.152709,0.678631
7,No log,1.078405,0.256944,0.148594,0.188295,0.688648
8,No log,1.027969,0.271429,0.152610,0.195373,0.698664
9,No log,0.980238,0.283465,0.144578,0.191489,0.703673
10,No log,0.938653,0.316239,0.148594,0.202186,0.711185


/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


TrainOutput(global_step=1600, training_loss=0.7387796902656555, metrics={'train_runtime': 2452.3187, 'train_samples_per_second': 10.194, 'train_steps_per_second': 0.652, 'total_flos': 170977533622896.0, 'train_loss': 0.7387796902656555, 'epoch': 50.0})

In [72]:
# Save the fine-tuned model
model.save_pretrained("/content/drive/MyDrive/fine-tuning-project/task3")
tokenizer.save_pretrained("/content/drive/MyDrive/fine-tuning-project/task3")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


('/content/drive/MyDrive/fine-tuning-project/task3/tokenizer_config.json',
 '/content/drive/MyDrive/fine-tuning-project/task3/special_tokens_map.json',
 '/content/drive/MyDrive/fine-tuning-project/task3/vocab.txt',
 '/content/drive/MyDrive/fine-tuning-project/task3/added_tokens.json',
 '/content/drive/MyDrive/fine-tuning-project/task3/tokenizer.json')

In [79]:
from transformers import AutoTokenizer, AutoModelForTokenClassification
from peft import PeftModel, PeftConfig

# Load the model and tokenizer
model = AutoModelForTokenClassification.from_pretrained("distilbert/distilbert-base-multilingual-cased", num_labels=len(label_list))
tokenizer = AutoTokenizer.from_pretrained("distilbert/distilbert-base-multilingual-cased")

# Verify and correct the path to the LoRA adapters
adapter_path = "/content/drive/MyDrive/fine-tuning-project/task3"  # Double-check this path
peft_config = PeftConfig.from_pretrained(adapter_path)

model = PeftModel.from_pretrained(model, adapter_path,  load_in_8bit=True, device_map='auto')

Some weights of DistilBertForTokenClassification were not initialized from the model checkpoint at distilbert/distilbert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [81]:
id2label = {i: label for i, label in enumerate(label_list)}
label2id = {label: i for i, label in enumerate(label_list)}

# Ensure the label mappings are set correctly
model.config.id2label = id2label
model.config.label2id = label2id

In [ ]:
def visualize_predictions(text, predictions):
    tokens = tokenizer.tokenize(text)
    for token, tag in zip(tokens, predictions):
        print(f"{token} -> {tag}")


In [85]:
import torch

# Define a sample input
sample_input = "باشگاه ورزشی زمالک"

# Tokenize the input
inputs = tokenizer(sample_input, return_tensors="pt")

# Make predictions
model.eval()
with torch.no_grad():
    outputs = model(**inputs)
logits = outputs.logits
predictions = torch.argmax(logits, dim=2)

# Convert predictions to NER tags
predicted_tags = [model.config.id2label[p.item()] for p in predictions[0]]

# Visualization function
def visualize_predictions(text, predictions):
    tokens = tokenizer.tokenize(text)
    for token, tag in zip(tokens, predictions):
        print(f"{token} -> {tag}")

# Visualize the output
visualize_predictions(sample_input, predicted_tags)


باشگاه -> I-ORG
و -> I-ORG
##رزش -> I-ORG
##ی -> I-ORG
ز -> I-ORG
##مال -> I-ORG
##ک -> I-ORG
